# Teste completo

## Fase 1:
- Divisão treino/teste com `random_state=1024`
- Normalização: z-score

## Fase 2:
- Balanceamento: smote

## Fase 3:
- Hiperparâmetros: pycaret
    - Regressão Logística
    - Floresta Aleatória


In [ ]:
nome_teste =  "TB"

In [ ]:
random_state = 1024

In [ ]:
# Permite acesso ao GDrive

In [ ]:
# Caminho do arquivo no GDrive
path_base = ''
#Onde salvar os arquivos temporários
dir_data = f"data_{nome_teste}"
dir_model = f"model_{nome_teste}"

# Fase 1: Processamento dos Dados

Este arquivo realiza o **pré-processamento** dos dados, preparando-os para as fases subsequentes de modelagem e análise. Abaixo está um resumo das principais etapas e funcionalidades implementadas nesta etapa.

## Etapas do Processamento

### 1. Importação das Bibliotecas
As bibliotecas necessárias para o processamento dos dados são importadas no início do arquivo, como:
- `pandas` para manipulação de dados
- `numpy` para operações numéricas

### 2. Carregamento do Dataset
O arquivo carrega os dados a partir de um arquivo CSV utilizando `pandas`


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
#cria diretórios
try:
    os.mkdir(dir_data)
    print(f"Directory '{dir_data}' created successfully.")
except FileExistsError:
    print(f"Directory '{dir_data}' already exists.")
except PermissionError:
    print(f"Permission denied: Unable to create '{dir_data}'.")
except Exception as e:
    print(f"An error occurred: {e}")
try:
    os.mkdir(dir_model)
    print(f"Directory '{dir_model}' created successfully.")
except FileExistsError:
    print(f"Directory '{dir_model}' already exists.")
except PermissionError:
    print(f"Permission denied: Unable to create '{dir_model}'.")
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
data = pd.read_csv('../data/heart_2022_with_nans.csv')

## Substituir valores nulos

In [ ]:
# Substituir valores nulos nas variáveis numéricas pela média
data.fillna(data.select_dtypes(include='number').mean(), inplace=True)

# Substituir valores nulos nas variáveis categóricas pela moda
data.fillna(data.select_dtypes(include='object').mode().iloc[0], inplace=True)

## Transformação de variáveis categóricas em numéricas

In [ ]:
# Cópia do dataframe
data_copy = data.copy()
data_copy.head()

In [ ]:
# Lista de colunas de Sim ou Não a serem mapeadas
yes_no_columns = [
    'PhysicalActivities', 'HadHeartAttack', 'HadAngina', 'HadStroke',
    'HadAsthma', 'HadSkinCancer', 'HadCOPD', 'HadDepressiveDisorder',
    'HadKidneyDisease', 'HadArthritis', 'DeafOrHardOfHearing',
    'BlindOrVisionDifficulty', 'DifficultyConcentrating',
    'DifficultyWalking', 'DifficultyDressingBathing',
    'DifficultyErrands', 'ChestScan', 'AlcoholDrinkers',
    'HIVTesting', 'FluVaxLast12', 'PneumoVaxEver',
    'HighRiskLastYear'
]

# Mapeando "Yes" para 1 e "No" para 0
data_copy[yes_no_columns] = data_copy[yes_no_columns].replace({'Yes': 1, 'No': 0})  #.astype(int) para converter para inteiro após remoção de valores nulos

In [ ]:
# Dicionário de mapeamento coluna GeneralHealth
health_mapping = {
    'Excellent': 5,
    'Very good': 4,
    'Good': 3,
    'Fair': 2,
    'Poor': 1
}

data_copy['GeneralHealth'] = data_copy['GeneralHealth'].map(health_mapping)

In [ ]:
# Mapeamento coluna Sex
sex_mapping = {
    'Male': 0,
    'Female': 1
}

data_copy['Sex'] = data_copy['Sex'].map(sex_mapping)

In [ ]:
# Mapeamento coluna RemovedTeeth
removed_teeth_mapping = {
    'None of them': 0,
    '1 to 5': 1,
    '6 or more, but not all': 2,
    'All': 3
}

data_copy['RemovedTeeth'] = data_copy['RemovedTeeth'].map(removed_teeth_mapping)

In [ ]:
# Mapeamento coluna AgeCategory
age_mapping = {
    'Age 18 to 24': 0,
    'Age 25 to 29': 1,
    'Age 30 to 34': 2,
    'Age 35 to 39': 3,
    'Age 40 to 44': 4,
    'Age 45 to 49': 5,
    'Age 50 to 54': 6,
    'Age 55 to 59': 7,
    'Age 60 to 64': 8,
    'Age 65 to 69': 9,
    'Age 70 to 74': 10,
    'Age 75 to 79': 11,
    'Age 80 or older': 12
}

data_copy['AgeCategory'] = data_copy['AgeCategory'].map(age_mapping)

In [ ]:
# Mapeamento coluna TetanusLast10Tdap
tetanus_mapping = {
    'No, did not receive any tetanus shot in the past 10 years': 0,
    'Yes, received tetanus shot but not sure what type': 1,
    'Yes, received Tdap': 2,
    'Yes, received tetanus shot, but not Tdap': 3
}

data_copy['TetanusLast10Tdap'] = data_copy['TetanusLast10Tdap'].map(tetanus_mapping)

In [ ]:
# Mapeamento da coluna LastCheckupTime
checkup_mapping = {
    "Within past year (anytime less than 12 months ago)": 0,
    "Within past 2 years (1 year but less than 2 years ago)": 1,
    "Within past 5 years (2 years but less than 5 years ago)": 2,
    "5 or more years ago": 3
}

# Aplicar o mapeamento à coluna LastCheckupTime
data_copy['LastCheckupTime'] = data_copy['LastCheckupTime'].map(checkup_mapping)

In [ ]:
# Aplicar one-hot encoding
data_copy = pd.get_dummies(data_copy, columns=['SmokerStatus', 'ECigaretteUsage', 'RaceEthnicityCategory', 'CovidPos', 'HadDiabetes'], drop_first=True, dtype='int')

In [ ]:
if 'State' in data_copy.columns:
  data_copy = data_copy.drop('State', axis=1)

In [ ]:
data_copy.to_csv(f'{dir_data}/Fase1-output_processed_data.csv', index=False)

### Normalização

In [ ]:
df = pd.read_csv(f'{dir_data}/Fase1-output_processed_data.csv')

#### Normalização com z-score

In [ ]:
# apply normalization techniques 
for column in df.columns: 
    df[column] = (df[column] - df[column].mean()) / df[column].std()    

In [ ]:
data_copy.to_csv(f'{dir_data}/Fase1-output_processed_data_normalized.csv', index=False)

# Fase 2: Divisão dos Dados
Este arquivo realiza a **divisão dos dados**, preparando-os para as fases subsequentes de modelagem e treinamento dos algoritmos de aprendizado de máquina. Abaixo está um resumo das principais etapas e funcionalidades implementadas nesta fase.

## Etapas da Divisão dos Dados

### 1. Importação das Bibliotecas
As bibliotecas necessárias para a criação e avaliação dos modelos são importadas, como:
- `pandas` para manipulação de dados
- `sklearn.model_selection` para a função train_test_split, que realiza a divisão dos dados

### 2. Definição das Variáveis Independentes e Dependentes
As variáveis são definidas para a divisão:

- `X`: Contém as variáveis independentes, ou seja, todas as colunas exceto a variável alvo.
- `y`: Contém a variável dependente, que neste caso é a coluna que indica se houve um ataque cardíaco `HadHeartAttack`.

### 3. Divisão dos Dados
 Os dados são divididos em conjuntos de treino e teste para avaliar o desempenho dos modelos. Esta divisão é realizada utilizando a função `train_test_split` do sklearn.

- `Treino (X_train, y_train)`: Usado para treinar os modelos.
- `Teste (X_test, y_test)`: Usado para avaliar a performance do modelo.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import joblib
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv(f'{dir_data}/Fase1-output_processed_data_normalized.csv')

In [ ]:
# Definir as variáveis independentes (X) e dependentes (y)
X = data.drop(columns=['HadHeartAttack'])  # Todas as colunas exceto a variável alvo
y = data['HadHeartAttack']  # A variável alvo

In [ ]:
# Dividir os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Salvar os dados divididos
save_to = f'{dir_data}/Fase2-output_data_splits.pkl'
joblib.dump((X_train, X_test, y_train, y_test), save_to)
print(f"Dados divididos salvos em {save_to}")

In [ ]:
# Salvar os dados de teste
save_to = f'{dir_data}/Fase2-output_data_test.pkl'
joblib.dump((X_test, y_test), save_to)
print(f"Dados de teste salvos em {save_to}")

In [ ]:
sns.countplot(x=y_train)
plt.show()

# Ver proporção das classes
print(y_train.value_counts(normalize=True))

# Fase 3 - Balanceamento dos Dados

Este arquivo implementa o balanceamento das classes da variável-alvo para evitar que o modelo de aprendizado de máquina se torne enviesado para a classe majoritária. Para isso, foi utilizado o SMOTE (Synthetic Minority Over-sampling Technique), uma técnica que cria amostras sintéticas da classe minoritária, equilibrando assim a distribuição das classes no conjunto de dados.

## Etapas do Balanceamento dos Dados

### 1. **Carregamento dos Dados**  
O conjunto de dados de treino é carregado a partir do arquivo salvo na fase anterior, possibilitando a continuação do fluxo de preparação.

### 2. **Aplicação do SMOTE para Balanceamento das Classes**
Foi utilizado o SMOTE para gerar amostras sintéticas da classe minoritária no conjunto de treino. Essa técnica ajuda a balancear as classes, evitando que o modelo aprenda de forma enviesada para a classe mais frequente

### 3. **Verificação das Proporções das Classes**
Após a aplicação do SMOTE, foi exibido as novas proporções das classes no conjunto de dados de treino para confirmar o balanceamento

### 4. Salvando Dados Balanceados
Por fim, os dados balanceados são salvos em um arquivo para serem utilizados na próxima fase de treinamento e teste dos modelos.

In [ ]:
import pandas as pd
from imblearn.over_sampling import SMOTE
import joblib

In [ ]:
# Carregar os dados de treino e teste
X_train, X_test, y_train, y_test = joblib.load(f'{dir_data}/Fase2-output_data_splits.pkl')

# Aplicar SMOTE
smote = SMOTE(random_state=random_state)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Exibir as novas proporções
print("Proporções após SMOTE:")
print(y_resampled.value_counts(normalize=True))

# Salvar os dados reamostrados
joblib.dump((X_resampled, y_resampled), f'{dir_data}/Fase3-output_data_resampled.pkl')

In [ ]:
!date

In [ ]:
from pycaret.classification import setup
from pycaret.classification import compare_models
from pycaret.classification import automl
from pycaret.classification import tune_model

import joblib
from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
# Carregar os dados de treino balanceados
# X_resampled, y_resampled = joblib.load('/content/drive/MyDrive/Orientacoes/Orientacoes/2024/Heloisa/data/Fase3-output_data_resampled.pkl')
X_resampled, y_resampled = joblib.load(f'{dir_data}/Fase3-output_data_resampled.pkl')

# Carregar os dados de teste
# X_test, y_test = joblib.load('/content/drive/MyDrive/Orientacoes/Orientacoes/2024/Heloisa/data/Fase2-output_data_test.pkl')
X_test, y_test = joblib.load(f'{dir_data}/Fase2-output_data_test.pkl')

# Concatena a base de treino em um único dataframe
train_data = pd.concat([X_resampled, y_resampled], axis=1)

# Concatena a base de teste em um único dataframe
test_data = pd.concat([X_test, y_test], axis=1)

# Concatena tudo em uma única base
data = pd.concat([train_data, test_data], ignore_index=True)

In [ ]:
train_data_pycaret = setup(data = train_data, target = 'HadHeartAttack', use_gpu=False)

In [ ]:
%%time
models = compare_models(include = ['rf', 'lr'])

In [ ]:
%%time
#Picking the winner
best_model = automl(optimize = 'Accuracy')

#Fine-tuning the best model
tuned_best_model = tune_model(best_model)

In [ ]:
best_model


In [ ]:
print("Fim.")

In [ ]:
!date

## Teste do modelo com os dados de teste

In [ ]:
# Testa o modelo com os dados de teste
from pycaret.classification import predict_model
from pycaret.classification import plot_model
from sklearn.metrics import confusion_matrix
y_predict = predict_model(best_model, data=X_test)

### Matriz de confusão

In [ ]:
cm = confusion_matrix(y_test, y_predict['prediction_label'])

In [ ]:
df_cm = pd.DataFrame(cm)
df_cm['total'] = df_cm[0] + df_cm[1]
df_cm['percent_0'] = df_cm[0] / df_cm['total'] * 100
df_cm['percent_1'] = df_cm[1] / df_cm['total'] * 100
df_cm.round(2)

### Salvando o modelo

In [ ]:
from pycaret.classification import save_model
from pycaret.classification import load_model

In [ ]:
#Salvando o modelo

save_model(best_model, f'{dir_model}/best_model')

In [ ]:
# functional API
loaded_model = load_model(f'{dir_model}/best_model')
print(loaded_model)

## Feature importance

In [ ]:
plot_model(best_model, plot='feature')